In [1]:
import json
default_survey = json.load(open("default_survey.qsf"))
default_question = default_survey['SurveyElements'][-1]
default_survey['SurveyElements'] = default_survey['SurveyElements'][:-1]
default_block = default_survey['SurveyElements'][0]['Payload'][0]
default_survey['SurveyElements'][0]['Payload'] = [default_survey['SurveyElements'][0]['Payload'][-1]]

In [2]:
def make_question_block(question_id):
    block_id = f'BL_{question_id}'
    flow_id = f'FL_{question_id}'
    
    new_block = copy.deepcopy(default_block)
    new_block['ID'] = block_id
    new_block['Type'] = 'Standard'
    new_block['Description'] = block_id
    new_block['BlockElements'][0]['QuestionID'] = question_id

    default_survey['SurveyElements'][1]['Payload']['Flow'].append({'ID': block_id, 'Type': 'Block', 'FlowID': flow_id})
    return new_block

def make_question(qid, qtext):
    questionId = f'QID{qid}'
    questionDescription = qtext.replace("<br>", " ")[:97] + '...'

    new_question = copy.deepcopy(default_question)
    new_question['PrimaryAttribute'] = questionId
    new_question['SecondaryAttribute'] = questionDescription
    new_question['Payload']['QuestionText'] = qtext
    new_question['Payload']['DataExportTag'] = f'Q{qid}'
    new_question['Payload']['QuestionID'] = questionId
    new_question['Payload']['QuestionDescription'] = questionDescription

    return new_question

In [3]:
attention_question = r"These sentences have the same structure:<br><br>For this sentence, ignore what we asked you for and choose 3.<br><br>For this clock, press what we told you for and climb 3.<br>"

In [4]:
import csv, copy, random

with open("devset_wrong.csv") as wrong:
    reader = csv.DictReader(wrong)
    wrong_questions = random.sample(list(reader), 25)

with open("devset.csv") as file:
    reader = csv.DictReader(file)
    questions = list(reader)
    random.shuffle(questions)


In [5]:
def make_survey(q, wq):
    default_survey = json.load(open("default_survey.qsf"))
    default_question = default_survey['SurveyElements'][-1]
    default_survey['SurveyElements'] = default_survey['SurveyElements'][:-1]
    default_block = default_survey['SurveyElements'][0]['Payload'][0]
    default_survey['SurveyElements'][0]['Payload'] = [default_survey['SurveyElements'][0]['Payload'][-1]]

    questions = q + wq
    random.shuffle(questions)
    for index, row in enumerate(questions):
        qid = index + 1
        qtext = fr"These sentences have the same structure:<br><br>{row['Original Sentence']}<br><br>{row['New Sentence']}<br>"
        new_question = make_question(qid, qtext)
        
        default_survey['SurveyElements'].append(new_question)
        default_survey['SurveyElements'][0]['Payload'].append(make_question_block(f'QID{qid}'))

        if index == 9:
            new_question = make_question(1000, attention_question)
            default_survey['SurveyElements'].append(new_question)
            default_survey['SurveyElements'][0]['Payload'].append(make_question_block('QID1000'))
            
    return default_survey

In [6]:
import tqdm
for i in tqdm.trange(5):
    with open(f"survey{i+1}.qsf", "w") as file:
        q = questions[i * 20 : i * 20 + 20]
        wq = wrong_questions[i * 5 : i * 5 + 5]
        survey = make_survey(q, wq)
        file.write(json.dumps(survey))

100%|████████████████████████████████████████████| 5/5 [00:00<00:00, 858.36it/s]
